In [1]:
#Loading Packages
import os
import csv
import requests
from bs4 import BeautifulSoup
import sys
import pandas as pd
from datetime import datetime
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
now = datetime.now()
date = now.date()
import re
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#Creating path for data extraction
now = datetime.now()
extraction_date = now.date()
date = extraction_date.strftime("%Y%m%d")
path = 'C:/Users/JenarthananRajenthir/OneDrive - ADA Asia/Documents/Final_telco/Data/MY/Hotlink/'             
if not os.path.exists(path):
    os.makedirs(path)  

In [3]:
#All device URLs
urls = ['https://store.hotlink.com.my/devicedetails/category/mobiles/vivo/y15s',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/realme/c21y',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/samsung/galaxy-m12',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/vivo/y15a',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/realme/8-5g',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/realme/c25s',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/oppo/a54',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/lenovo/tab-m10-hd-(2nd-gen)',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/samsung/galaxy-tab-a-(8-0-inch-2019)',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/vivo/y20-2021',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/xiaomi/redmi-10',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/samsung/galaxy-a03s',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/realme/c11',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/realme/c21',
        'https://store.hotlink.com.my/devicedetails/category/mobiles/vivo/y12s']
        

In [7]:
#Extraction of Devices Data
for url in urls:
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument("--disable-extensions")
    options.add_argument("--proxy-server='direct://'")
    options.add_argument("--proxy-bypass-list=*")
    options.add_argument("--start-maximized")
    options.add_argument('--disable-gpu')
    options.add_argument("--incognito")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--no-sandbox')
    options.add_argument("--disable-blink-features=AutomationControlled")
    browser = webdriver.Chrome(executable_path='../../chromedriver.exe', options=options)
    browser.get(url)
    html_source_code = browser.execute_script("return document.innerHTML;")
    browser.set_page_load_timeout(10)

    html = browser.page_source
    soup = BeautifulSoup(html, "html5lib")
    browser.implicitly_wait(40)

#     close = browser.find_element(By.XPATH,"//div[@class='sp-fancybox-overlay']")
#     browser.execute_script("arguments[0].click();", close)

    page_body_height = browser.execute_script("return document.body.scrollHeight")
    scroll_to = page_body_height / 4
    browser.execute_script(f"window.scrollTo(0, {scroll_to})")

    colours = browser.find_elements(By.XPATH,"//div[@id='colors-section']//child::span")

    df_final = pd.DataFrame(columns=['Device_Name','Device_Brand','Extraction_Date','Contract_Option', 'Colour', 'Storage_Size', 'Package_Name','Monthly_Device_Instalment',
                                     'Plan_Monthly_Charges','Device_Advance_Payment', 'Upfront_Payment', 'Device_Payment_Period','Device_Advance_Refunded_After_12months'])

    for colour in colours:
        browser.execute_script("arguments[0].click();", colour)
        browser.implicitly_wait(10)
        storages = browser.find_elements(By.XPATH,"//div[@id='storage-section']//child::p")
        for storage in range(1,len(storages)): 
            #print(storage)
            browser.execute_script("arguments[0].click();", storages[storage])
            browser.implicitly_wait(10)
            contracts_text = [x.get_text() for x in soup.find_all('div', attrs = {'id':'devicePlan'})[0].find_all('div', attrs = {'class':'row mb-3'})[0].find_all('p')]
            for contract_text in reversed(contracts_text): 
                staleElement = True
                while staleElement:
                    try:
                        #print("try")
                        contracts = browser.find_elements(By.XPATH,"//p[contains(text(),'"+contract_text+"')]")
                        browser.execute_script("arguments[0].click();", contracts[0])
                        staleElement = False
                    except:
                        #print("except")
                        staleElement =True
                browser.implicitly_wait(10)
                html_source_code = browser.execute_script("return document.innerHTML;")
                browser.set_page_load_timeout(10)

                html = browser.page_source
                soup = BeautifulSoup(html, "html5lib")
                browser.implicitly_wait(10)
                #print(contract_text)
                try:
                    dev_name = soup.find_all('div', attrs = {'id':'device-details'})[0].find_all('div', attrs = {'class':'react-label container max-md-width d-none d-md-block'})[0].find_all('p')[0].text
                    dev_brand = soup.find_all('div', attrs = {'id':'device-details'})[0].find_all('p', attrs = {'class':'fs-16 text-steel maxis mb-0 mt--8px mt-md-0 text-uppercase'})[0].text
                    col = soup.find_all('div', attrs = {'class':'py-3 px-0 px-md-3'})[0].find_all('p')[1].text
                    storage = soup.find_all('div', attrs = {'id':'storage-section'})[0].find_all('div', attrs = {'class':'align-self-center'})[0].text
                    cont_op = contract_text
                    pack_name = soup.find_all('div', attrs = {'class':'col-5 cart-details'})[0].find_all('p', attrs = {'class':'maxis-extraBold fs-16 m-0'})[1].text
                    month_dev_ins = soup.find_all('div', attrs = {'class':'col-5 cart-details'})[0].find_all('div', attrs = {'class':'col-5 ml-auto maxis-black fs-22'})[0].find_all('span')[0].text
                    plan_month_charge = soup.find_all('div', attrs = {'class':'col-5 cart-details'})[0].find_all('p', attrs = {'class':'fs-22 maxis-black m-0'})[0].find_all('span')[0].text
                    dev_advance_pay = soup.find_all('div', attrs = {'class':'col-5 cart-details'})[0].find_all('p', attrs = {'class':'fs-20 maxis-black m-0'})[0].find_all('span')[0].text
                    upfront_pay = soup.find_all('div', attrs = {'class':'col-auto pr-3 ml-auto lh-130 text-right'})[0].find_all('p', attrs = {'class':'fs-32 maxis-black m-0'})[0].find_all('span')[0].text
                    dev_pay_period = soup.find_all('div', attrs = {'class':'col-5 cart-details'})[0].find_all('span')[4].text
                    device_advance = soup.find_all('div', attrs = {'class':'col-5 cart-details'})[0].find_all('p', attrs = {'class':'fs-20 maxis-black m-0'})[0].find_all('span')[0].text


                    df_final = df_final.append({'Device_Name': dev_name, 'Device_Brand':dev_brand,'Extraction_Date': extraction_date,'Contract_Option': cont_op, 'Colour':col, 'Storage_Size': storage, 'Package_Name':pack_name,'Monthly_Device_Instalment': month_dev_ins,
                                            'Plan_Monthly_Charges': plan_month_charge,'Device_Advance_Payment': dev_advance_pay, 'Upfront_Payment': upfront_pay, 'Device_Payment_Period': dev_pay_period,
                                           'Device_Advance_Refunded_After_12months':device_advance}, ignore_index=True)
                except:
                    continue
    df_final.to_csv(path+date+'_'+dev_name+'_devices.csv', index=False)
    print('Successfully Hotlink '+dev_name+' Details Collected!')
    browser.quit()

<ipython-input-7-58e325e43a6e>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='../../chromedriver.exe', options=options)


Successfully Hotlink Y15s Details Collected!
Successfully Hotlink C21Y Details Collected!
Successfully Hotlink Galaxy M12 Details Collected!
Successfully Hotlink Y15A Details Collected!
Successfully Hotlink Y15A Details Collected!
Successfully Hotlink Y15A Details Collected!
Successfully Hotlink Y15A Details Collected!
Successfully Hotlink Tab M10 HD Details Collected!
Successfully Hotlink Tab M10 HD Details Collected!
Successfully Hotlink Tab M10 HD Details Collected!
Successfully Hotlink Redmi 10 Details Collected!
Successfully Hotlink Redmi 10 Details Collected!
Successfully Hotlink C11 2021 Details Collected!
Successfully Hotlink C11 2021 Details Collected!
Successfully Hotlink C11 2021 Details Collected!
